In [1]:
import sys
import pandas as pd
import numpy as np
import datetime
import glob
import gc
import os

#========================================================================
# Args
#========================================================================
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month']

win_path = f'../features/4_winner/*.gz'
fname=''
# submit = pd.read_csv('../input/sample_submission.csv')
submit = []

HOME = os.path.expanduser('~')

sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from preprocessing import get_ordinal_mapping
from utils import logger_func
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

import torch
import matplotlib.pyplot as plt
from torch import nn, optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

#========================================================================
# Data Load
base = utils.read_df_pkl('../input/base*')
win_path_list = glob.glob(win_path)
train_path_list = []
test_path_list = []
for path in win_path_list:
    if path.count('train'):
        train_path_list.append(path)
    elif path.count('test'):
        test_path_list.append(path)

base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)
train_feature_list = utils.parallel_load_data(path_list=train_path_list)
test_feature_list = utils.parallel_load_data(path_list=test_path_list)
train = pd.concat(train_feature_list, axis=1)
train = pd.concat([base_train, train], axis=1)
test = pd.concat(test_feature_list, axis=1)
test = pd.concat([base_test, test], axis=1)

# y = train[[key, target]]
y = train[target]
# y[target] = y[target].map(lambda x: 1 if x<-30 else 0)
train.drop(target, axis=1, inplace=True)
test.drop(target, axis=1, inplace=True)

train_test = pd.concat([train, test], axis=0)
train_test.set_index(key, inplace=True)
train_test.fillna(train_test.median(), inplace=True)

print(train.shape)

2019-01-05 10:40:19,061 utils 366 [INFO]    [logger_func] start 
100%|██████████| 3/3 [00:00<00:00, 60.13it/s]


(201917, 155)


Process ForkPoolWorker-69:
Process ForkPoolWorker-68:
Process ForkPoolWorker-67:
Traceback (most recent call last):
Process ForkPoolWorker-66:
Traceback (most recent call last):
Process ForkPoolWorker-65:
Process ForkPoolWorker-64:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-33:
Process ForkPoolWorker-2:
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-63:
Process ForkPoolWorker-35:
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-3:
Process ForkPoolWorker-36:
Process ForkPoolWorker-4:
Process ForkPoolWorker-60:
Process ForkPoolWorker-37:
Process ForkPoolWorker-5:
Process ForkPoolWorker-57:
Process ForkPoolWorker-25:
Process ForkPoolWorker-56:
Process ForkPoolWorker-24:
Process ForkPoolWorker-55:
Process ForkPoolWorker-23:
Process ForkPoolWorker-22:
Process ForkPoolW

In [2]:
columns = train_test.columns
scaler = MinMaxScaler()
train_test = pd.DataFrame(scaler.fit_transform(train_test), columns = columns, index=train_test.index)
display(train_test.head())

train = train_test.iloc[:len(train)]
test = train_test.iloc[len(train):]
print(train.shape)
print(test.shape)

,111_ker_card_id_total@,111_ker_elapsed_time@,111_ker_feature_1@,111_ker_feature_2@,111_ker_feature_3@,111_ker_hist_authorized_flag_mean@,111_ker_hist_authorized_flag_sum@,111_ker_hist_card_id_size@,111_ker_hist_dayofweek_nunique@,111_ker_hist_first_buy@,...,118_ah0_auth0_hist_category_3_mean_mean@,118_ah0_auth0_hist_month_diff_mean@,118_ah0_auth0_hist_purchase_amount_max@,118_ah0_auth0_hist_purchase_amount_mean@,118_ah0_auth0_hist_purchase_amount_sum@,119_ath_auth1_0_hist_purchase_date_max_diff@,119_ath_auth1_0_hist_purchase_date_min_diff@,130_ddf_hist_last25_date_diff_std@,132_ddf_hist_first10_diff_days_lag1@_std@,132_ddf_hist_last5_diff_days_lag1@_mean@
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,0.050358,0.107268,1.000000,0.000000,1.0,0.948428,0.059466,0.046237,1.0,0.143836,...,0.713795,0.011834,5.240250e-08,1.456250e-08,0.000167,0.512718,0.515385,0.035240,0.001919,0.003089
C_ID_3d0044924f,0.063441,0.173380,0.521635,0.486242,0.0,0.967583,0.081796,0.062366,1.0,0.135375,...,0.025394,0.097902,4.474975e-07,1.072791e-07,0.000168,0.479250,0.466667,0.015864,0.004587,0.000772
C_ID_d639edf6cd,0.007527,0.240368,0.501639,0.000000,0.0,0.952026,0.009466,0.007348,1.0,0.199033,...,1.000000,0.000000,1.819750e-08,1.287875e-08,0.000168,0.851406,0.483333,0.457728,0.025445,0.090347
C_ID_186d6a6901,0.014695,0.066988,0.521635,1.000000,0.0,1.000000,0.018204,0.013441,1.0,0.143433,...,0.312486,0.015385,4.124750e-08,1.885420e-08,0.000168,0.473896,0.505128,0.101347,0.003161,0.007722
C_ID_cdbd2c0db2,0.029928,0.040280,0.000000,1.000000,0.0,0.961224,0.030583,0.023477,1.0,0.137792,...,0.027933,0.000000,1.320987e-06,8.014925e-07,0.000172,0.435074,0.450000,0.029219,0.004411,0.000000


(201917, 154)
(123623, 154)


(161533, 154)
(40384, 154)


In [17]:
class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(154, 1)
#         self.fc2 = nn.Linear(77, 30)
#         self.fc2 = nn.Linear(10, 1)
        self.dropout = nn.Dropout(p=0.1)
        
    def forward(self, x):
        
        x = self.dropout(self.fc1(x))
#         x = self.dropout(self.fc2(x))
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
        
        return x
    
x_train, x_val, y_train, y_val = train_test_split(train, y, test_size=0.2, random_state=1208)
print(x_train.shape)
print(x_val.shape)
x_train.head()
    
batch_size = 50
train_batch = np.array_split(x_train, batch_size)
label_batch = np.array_split(y_train, batch_size)

for i in range(len(train_batch)):
    train_batch[i] = torch.from_numpy(train_batch[i].values).float()
for i in range(len(label_batch)):
    label_batch[i] = torch.from_numpy(label_batch[i].values).float().view(-1, 1) # (24)を(24,1)にする。

x_val = torch.from_numpy(x_val.values).float()
y_val = torch.from_numpy(y_val.values).float().view(-1, 1) # 1次元Tensorじゃだめなんだ

model = Regressor()
ps = model(train_batch[0])
ps.shape

(161533, 154)
(40384, 154)


torch.Size([3231, 1])

In [18]:
model = Regressor()
# Loss Function
criterion = nn.MSELoss()
# Optimize Function
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 300

train_losses, test_losses = [], []

for e in range(epochs):
    model.train()
    train_loss = 0
    
    for i in range(len(train_batch)):
        # 勾配情報の初期化
        optimizer.zero_grad()
        # バッチ毎に学習を行う
        output = model(train_batch[i])
        print(np.unique(output.detach().numpy()))
        # バッチの損失
        loss = torch.sqrt(criterion(torch.log(output), torch.log(label_batch[i])))
        # 逆伝播
        loss.backward()
        # 勾配情報の更新
        optimizer.step()
        
        train_loss += loss.item()
        
    else:
        test_loss = 0
        accuracy = 0
        
        with torch.no_grad():
            model.eval()
            predictions = model(x_val)
            test_loss += torch.sqrt(criterion(torch.log(predictions), torch.log(y_val)))
            
        train_losses.append(train_loss/len(train_batch))
        test_losses.append(test_loss)
        
        print(f"Epoch: {e+1}/{epochs}.. Training Loss: {train_loss/len(train_batch)}.. Test Loss: {test_loss}.. ")

[-0.6284856  -0.6087742  -0.60105014 ...  0.35821614  0.3647997
  0.4248745 ]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
[nan nan nan ... nan n

KeyboardInterrupt: 

In [ ]:
np.unique(output.to_numpy)

### Pytorch 実装ハンドブック

In [ ]:
import torch
import torch.nn as nn

# 各シーケンス長
seq_len_list = [8, 6, 10]

input_size = 2
hidden_size = 4

# Input Data Sample
input_seq = []
for seq_len in seq_len_list:
    iseq = torch.randn(seq_len, input_size)
    print('input sequence: ', iseq.shape)
    input_seq.append(iseq)
    
# Network Definition
net = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True)

# Forward
output_seq = []
for iseq in input_seq:
    iseq = iseq.unsqueeze(0) # 先頭にバッチサイズの次元1を加える
    print("input sequence add batch_size index 0", iseq.shape)
    oseq, _ = net(iseq)
    output_seq.append(oseq.squeeze(0)) # squeezeで先頭のバッチサイズの次元1を削除してリストに追加
    
# Check Output Data    
for oseq in output_seq:
    print("output sequence:", oseq.shape)
    
    
#========================================================================

batch_size = 20
seq_len = 10

input_size = 2
hidden_size = 4

# Input Data Sample
input_seq = torch.randn(batch_size, seq_len, input_size)
print("Input Sequence:", input_seq.shape)

# Network Definition
net = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True)

# 順伝搬
output_seq, _ = net(input_seq)

# 出力データの確認
print("output sequence:", output_seq.shape)

#========================================================================

batch_size = 20
seq_len = 10

input_size = 2
hidden_size = 4

# Input Data Sample
input_seq = torch.randn(batch_size, seq_len, input_size)
print("input sequence:", input_seq.size())

# Network Definition
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # LSTM Block A
        self.block_a = nn.LSTMCell(input_size, hidden_size)
        # LSTM Block B
        self.block_b = nn.LSTMCell(hidden_size, hidden_size)
        
    def forward(self, x, hx_a0, cx_a0, hx_b0, cx_b0):
        '''
        :param x: 現ステップの入力シーケンスの状態
        :param hx_a0: 前ステップのLSTM Block Aの隠れ層の状態
        :param cx_a0: 前ステップのLSTM Block Aのセル状態
        :param hx_b0: 前ステップのLSTM Block Bの隠れ層の状態
        :param cx_b0: 前ステップのLSTM Block Bのセル状態
        :return: 現ステップのLSTMブロックAの隠れ層の状態とセル状態、現ステップのLSTMブロックBの隠れ層の状態とセル状態
        '''
        
        hx_a1, cx_a1 = self.block_a(x, (hx_a0, cx_a0)) # 現ステップのLSTMブロックAの中間層の状態とセル状態
        hx_b1, cx_b1 = self.block_b(hx_a1, (hx_b0, cx_b0)) # 現ステップのLSTMブロックBの中間層の状態とセル状態
        return hx_a1, cx_a1, hx_b1, cx_b1
        
net = Net()

# 中間層の初期化
hx_a = torch.randn(batch_size, hidden_size)
cx_a = torch.randn(batch_size, hidden_size)
hx_b = torch.randn(batch_size, hidden_size)
cx_b = torch.randn(batch_size, hidden_size)

# 順伝搬
output_seq = []
for i in range(seq_len):
    hx_a, cx_a, hx_b, cx_b = net(input_seq[:, i, :], hx_a, cx_a, hx_b, cx_b)
    output_seq.append(hx_b)

# リストからTensorに変換
output_seq = torch.stack(output_seq, dim=1)

# Check Output Data
print("output sequence:", output_seq.shape)

#========================================================================

seq_len_list = [8, 6, 10]

input_size = 2
hidden_size = 4

# Input Data Sample
input_seq = []
for seq_len in seq_len_list:
    iseq = torch.randn(seq_len, input_size)
    print("input sequence:", iseq.shape)
    input_seq.append(iseq)
    
# Network Definition
net = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True)

# 順伝播
output_data = []

for iseq in input_seq:
    iseq = iseq.unsqueeze(0) # 先頭にバッチサイズの次元1を加える
    oseq, _ = net(iseq)
    
    # スライスでシーケンスの最終ステップのみ取り出し、squeezeで先頭のバッチサイズの次元1を除去してリストに追加
    output_data.append(oseq[:, -1, :].squeeze(0))
    
# check output data 
for odata in output_data:
    print("output data:", odata.shape)